In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Testing files/TCGAbiolinks/mutation_filtered.csv')
train.head()

,bcr_patient_barcode,A1CF,A2BP1,A2M,A2ML1,AACS,AADAC,AADACL4,AAK1,AARS,...,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZUFSP,ZWILCH,ZXDB,ZXDC,ZZEF1,ZZZ3
0,TCGA-A8-A0A6-01,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,1,1,0
1,TCGA-A8-A075-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TCGA-C8-A1HM-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TCGA-BH-A0BZ-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,TCGA-BH-A0BT-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
target=train['bcr_patient_barcode']

In [5]:
train.drop(['bcr_patient_barcode'], axis=1, inplace=True)
train.head()

,A1CF,A2BP1,A2M,A2ML1,AACS,AADAC,AADACL4,AAK1,AARS,AARS2,...,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZUFSP,ZWILCH,ZXDB,ZXDC,ZZEF1,ZZZ3
0,0,1,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_scaled = minmax_scale(train, axis = 0)

In [7]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [8]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 32

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(14, 7966)
(2, 7966)
Epoch 1/50
2/2 - 2s - loss: 0.6931 - val_loss: 0.6931 - 2s/epoch - 1s/step
Epoch 2/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 149ms/epoch - 75ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 129ms/epoch - 65ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 129ms/epoch - 64ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 134ms/epoch - 67ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 133ms/epoch - 66ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 163ms/epoch - 82ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 130ms/epoch - 65ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 130ms/epoch - 65ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 115ms/epoch - 57ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 156ms/epoch - 78ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6931 - val_loss: 0.6931 - 116ms/epoch - 58ms/step
Epoch 13/50
2/2 - 0s 

In [9]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0.000000,0.000000,0.199300,0.000000,0.000000,0.340379,0.338612,0.500280,0.000000,0.745151,...,0.256154,0.373102,0.724075,0.325620,0.287110,0.208774,0.169706,0.000000,0.000000,0.362280
1,0.000000,0.056034,0.073679,0.000000,0.000000,0.000000,0.018728,0.048416,0.000000,0.107906,...,0.022403,0.029223,0.082975,0.014116,0.059362,0.000000,0.000000,0.000000,0.001181,0.004114
2,0.081834,0.098517,0.036885,0.000000,0.000000,0.015477,0.184784,0.083142,0.041402,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.047848,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.136315,0.026534,0.020623,0.075373,0.000000,0.262750,0.135368,0.057827,0.072994,0.036251,...,0.000000,0.102629,0.085745,0.002498,0.000000,0.027436,0.083475,0.000000,0.007569,0.044933
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043549,0.020165,0.009593,0.015935,...,0.021771,0.000000,0.013658,0.000000,0.039052,0.024940,0.043152,0.000000,0.000000,0.000000
5,0.019782,0.000000,0.000000,0.023524,0.000000,0.057030,0.165941,0.000000,0.000000,0.071493,...,0.000000,0.055246,0.119294,0.000000,0.000000,0.000000,0.000000,0.000000,0.019586,0.048398
6,0.023753,0.010765,0.004136,0.027712,0.000000,0.058637,0.063002,0.000000,0.003187,0.034958,...,0.000000,0.018160,0.058357,0.029161,0.015238,0.027987,0.000000,0.000000,0.006710,0.000000
7,0.011718,0.000000,0.035694,0.000000,0.000000,0.000000,0.052462,0.012963,0.021427,0.078859,...,0.021662,0.025777,0.032538,0.000000,0.044717,0.000000,0.009475,0.000000,0.023093,0.000000
8,0.000000,0.000000,0.027555,0.000000,0.010833,0.000000,0.007940,0.042713,0.000000,0.082790,...,0.030435,0.076333,0.048171,0.000000,0.105727,0.000000,0.000000,0.000000,0.000000,0.013124
9,0.000000,0.003564,0.049450,0.000000,0.000000,0.017956,0.000000,0.023166,0.000000,0.072480,...,0.042968,0.072703,0.055472,0.000000,0.024167,0.000000,0.027263,0.000000,0.031196,0.019588


In [10]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7966)]            0         
                                                                 
 dense (Dense)               (None, 128)               1019776   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 7966)              102761

In [11]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()>0.75])
encoded_train.shape

(16, 27)

In [13]:
#encoded_train.to_excel('/content/drive/MyDrive/Testing files/TCGAbiolinks/dae_mutation.xlsx', index=False)